# 1 Import Library

In [1]:
import requests
import time
import pandas as pd

# 2 Parameter Setting

In [2]:
# 2-1) patent keyword
searchText_f = ["autonomous","self-driving","driverless","robot"]
searchText_b = ["car","vehicle","drive","driver","driving"]
searchText_T = []
for f in searchText_f:
    for b in searchText_b:
        searchText_T.append(f+" "+b)

print(searchText_T)
print(len(searchText_T))

['autonomous car', 'autonomous vehicle', 'autonomous drive', 'autonomous driver', 'autonomous driving', 'self-driving car', 'self-driving vehicle', 'self-driving drive', 'self-driving driver', 'self-driving driving', 'driverless car', 'driverless vehicle', 'driverless drive', 'driverless driver', 'driverless driving', 'robot car', 'robot vehicle', 'robot drive', 'robot driver', 'robot driving']
20


In [3]:
# 2-2) greater than and equal to ~
patent_date = "2007-01-01"

# 2-3) ustpo api post url 
url_post = 'http://www.patentsview.org/api/patents/query'
# ref : http://www.patentsview.org/api/query-language.html 

# 2-4) post page size
post_len = 10000

# 3 Post_func definition

In [4]:
def post_data(page,per_page,patent_date,search_text):
    data = {
            'q':{
                "_and":[#1) date range
                        {"_gte":
                            {"patent_date":patent_date}},
                            {"_or":[
                                {"_text_all": #2) patent title
                                    {"patent_title":search_text}},
                                {"_text_all": #3) patent abstract
                                    {"patent_abstract":search_text}}
                                       ]}
                        ]
                    },
            'f':[ "patent_number", # 1) pat_num
                  "patent_title",  # 2) title
                  "patent_date",   # 3) pat_grant date
                  "ipc_section",   # 4) pat_ipc
                  "ipc_class",
                  "ipc_subclass",
                  "patent_type",    # 5) pat_type
                  "patent_abstract"
                  ],
            'o':{"page":page,
                 "per_page":per_page
                    }
            }
    return data

# 4 Patent Request

In [5]:
# post total page list
total_patent = []

# post data
for t,search_t in enumerate(searchText_T):
    print("search num: ",t)
    print("search text: ", search_t)
    st = time.time()
    
    pt_dict_list = []
    
    # post first page
    try:
        t_data = post_data(1,post_len,patent_date,search_t) 
        t_post = requests.post(url_post,json=t_data).json()
        print("total_patent_num :",t_post['total_patent_count'])
        pt_dict_list.extend(t_post['patents'])
        
        post_page = int(t_post['total_patent_count']/post_len)+2
        time.sleep(1)
        
    except:
        print("=== Try error1 ===")
    
    # post total page
    if post_page >2:                   
        for n in range(2,post_page):
            
            print("page number :",n,"/",post_page)
            try:
                data = post_data(n,post_len,patent_date,search_t)
                js_data_post = requests.post(url_post,json=data).json()
                pt_dict_list.extend(js_data_post['patents'])
            
            except:
                print("=== Try error2 ===")
            
            
    et = time.time()
    print("exe_time :",et-st)
    time.sleep(1)   
    total_patent.extend(pt_dict_list)

('search num: ', 0)
('search text: ', 'autonomous car')
('total_patent_num :', 14)
('exe_time :', 2.010999917984009)
('search num: ', 1)
('search text: ', 'autonomous vehicle')
('total_patent_num :', 1453)
('exe_time :', 6.878999948501587)
('search num: ', 2)
('search text: ', 'autonomous drive')
('total_patent_num :', 242)
('exe_time :', 2.6480000019073486)
('search num: ', 3)
('search text: ', 'autonomous driver')
('total_patent_num :', 217)
('exe_time :', 2.6419999599456787)
('search num: ', 4)
('search text: ', 'autonomous driving')
('total_patent_num :', 411)
('exe_time :', 3.072999954223633)
('search num: ', 5)
('search text: ', 'self-driving car')
('total_patent_num :', 61)
('exe_time :', 2.0530002117156982)
('search num: ', 6)
('search text: ', 'self-driving vehicle')
('total_patent_num :', 1253)
('exe_time :', 5.273000001907349)
('search num: ', 7)
('search text: ', 'self-driving drive')
('total_patent_num :', 1221)
('exe_time :', 5.317999839782715)
('search num: ', 8)
('searc

In [6]:
print(len(total_patent))

7391


# 5 Filter ipc list

In [7]:
# 1) filter util patent (not design etc..)
util_patent = []
for pat in total_patent:
    if pat['patent_type'] == 'utility':
          util_patent.append(pat)

print(util_patent[0])

{u'patent_date': u'2018-06-19', u'patent_type': u'utility', u'IPCs': [{u'ipc_section': u'G', u'ipc_class': u'7', u'ipc_subclass': u'B'}, {u'ipc_section': u'G', u'ipc_class': u'1', u'ipc_subclass': u'C'}, {u'ipc_section': u'G', u'ipc_class': u'5', u'ipc_subclass': u'D'}, {u'ipc_section': u'G', u'ipc_class': u'6', u'ipc_subclass': u'Q'}], u'patent_title': u'Method and system for providing autonomous car errands', u'patent_number': u'10002466', u'patent_abstract': u"A delivery server may generate two separate security codes following a transaction between a user and a vendor, as well as a hash of the combination of the codes. The server may provide the first code to the user and the second code to the vendor, and provide the hash of a combination of the codes to both the vendor and the user. The user's vehicle may receive mapping information for the vendor's location, and may use the mapping information to autonomously navigate to an exchange station at the vendor location. At the exchang

In [8]:
# 2) ipcs ->list
st = time.time()
for d in util_patent:
    ipc_list = []
    for c in d['IPCs']:
        # ==== ipc class prob ['g1c','g5d' ... ] ===== #
        if len(str(c['ipc_class']))==1:
            c['ipc_class'] = str('0') + c['ipc_class']

        ipc = c['ipc_section'] + str(c['ipc_class']) + c['ipc_subclass']
        ipc_list.append(ipc)
    d['ipc_list'] = sorted(ipc_list)

et = time.time()
print("exe_time :",et-st)

print(util_patent[0])

('exe_time :', 0.03699994087219238)
{u'patent_date': u'2018-06-19', u'patent_type': u'utility', u'IPCs': [{u'ipc_section': u'G', u'ipc_class': u'07', u'ipc_subclass': u'B'}, {u'ipc_section': u'G', u'ipc_class': u'01', u'ipc_subclass': u'C'}, {u'ipc_section': u'G', u'ipc_class': u'05', u'ipc_subclass': u'D'}, {u'ipc_section': u'G', u'ipc_class': u'06', u'ipc_subclass': u'Q'}], 'ipc_list': [u'G01C', u'G05D', u'G06Q', u'G07B'], u'patent_title': u'Method and system for providing autonomous car errands', u'patent_number': u'10002466', u'patent_abstract': u"A delivery server may generate two separate security codes following a transaction between a user and a vendor, as well as a hash of the combination of the codes. The server may provide the first code to the user and the second code to the vendor, and provide the hash of a combination of the codes to both the vendor and the user. The user's vehicle may receive mapping information for the vendor's location, and may use the mapping informat

# 6 Save to csv file

In [9]:
# dict to dataframes
df = pd.DataFrame.from_dict(util_patent).drop(['IPCs'],axis=1)

print(df.iloc[0])
print('='*50)
print(df['ipc_list'][0])

ipc_list                                    [G01C, G05D, G06Q, G07B]
patent_abstract    A delivery server may generate two separate se...
patent_date                                               2018-06-19
patent_number                                               10002466
patent_title       Method and system for providing autonomous car...
patent_type                                                  utility
Name: 0, dtype: object
[u'G01C', u'G05D', u'G06Q', u'G07B']


In [10]:
df.head()

,ipc_list,patent_abstract,patent_date,patent_number,patent_title,patent_type
0,"[G01C, G05D, G06Q, G07B]",A delivery server may generate two separate se...,2018-06-19,10002466,Method and system for providing autonomous car...,utility
1,"[G01C, G05D, G08G, H04B, H04W]","Methods, apparatuses, and non-transitory compu...",2018-10-30,10115305,Optimizing autonomous car's driving time and u...,utility
2,"[B61D, B61L, E01B]",An autonomous ballast consist for unloading ba...,2018-11-27,10138603,Autonomous ballast unloading consist,utility
3,[B61J],An active rail transport system for the automa...,2012-04-03,8146506,Active rail transport system,utility
4,"[A61M, G05D]",An autonomous vehicle is provided that include...,2015-07-14,9081382,Autonomous vehicle comprising extracorporeal b...,utility


In [11]:
len(df)

7329

In [12]:
k= "US"+ df.patent_number
df.patent_number=k

In [13]:
df.head()

,ipc_list,patent_abstract,patent_date,patent_number,patent_title,patent_type
0,"[G01C, G05D, G06Q, G07B]",A delivery server may generate two separate se...,2018-06-19,US10002466,Method and system for providing autonomous car...,utility
1,"[G01C, G05D, G08G, H04B, H04W]","Methods, apparatuses, and non-transitory compu...",2018-10-30,US10115305,Optimizing autonomous car's driving time and u...,utility
2,"[B61D, B61L, E01B]",An autonomous ballast consist for unloading ba...,2018-11-27,US10138603,Autonomous ballast unloading consist,utility
3,[B61J],An active rail transport system for the automa...,2012-04-03,US8146506,Active rail transport system,utility
4,"[A61M, G05D]",An autonomous vehicle is provided that include...,2015-07-14,US9081382,Autonomous vehicle comprising extracorporeal b...,utility


In [14]:
df = df.drop('patent_type', 1)

In [15]:
df.head()

,ipc_list,patent_abstract,patent_date,patent_number,patent_title
0,"[G01C, G05D, G06Q, G07B]",A delivery server may generate two separate se...,2018-06-19,US10002466,Method and system for providing autonomous car...
1,"[G01C, G05D, G08G, H04B, H04W]","Methods, apparatuses, and non-transitory compu...",2018-10-30,US10115305,Optimizing autonomous car's driving time and u...
2,"[B61D, B61L, E01B]",An autonomous ballast consist for unloading ba...,2018-11-27,US10138603,Autonomous ballast unloading consist
3,[B61J],An active rail transport system for the automa...,2012-04-03,US8146506,Active rail transport system
4,"[A61M, G05D]",An autonomous vehicle is provided that include...,2015-07-14,US9081382,Autonomous vehicle comprising extracorporeal b...


In [16]:
df.patent_date=df.patent_date.str[:4]

In [17]:
df.head()

,ipc_list,patent_abstract,patent_date,patent_number,patent_title
0,"[G01C, G05D, G06Q, G07B]",A delivery server may generate two separate se...,2018,US10002466,Method and system for providing autonomous car...
1,"[G01C, G05D, G08G, H04B, H04W]","Methods, apparatuses, and non-transitory compu...",2018,US10115305,Optimizing autonomous car's driving time and u...
2,"[B61D, B61L, E01B]",An autonomous ballast consist for unloading ba...,2018,US10138603,Autonomous ballast unloading consist
3,[B61J],An active rail transport system for the automa...,2012,US8146506,Active rail transport system
4,"[A61M, G05D]",An autonomous vehicle is provided that include...,2015,US9081382,Autonomous vehicle comprising extracorporeal b...


In [18]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
df.to_csv("selfdriving_patent_0329.csv")